#### Implementing early stopping

In [1]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

Using TensorFlow backend.


In [2]:
# Reading the data
train = pd.read_csv('age_detection_train/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('age_detection_train/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = utils.np_utils.to_categorical(train_y)

In [3]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 512

In [4]:
model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, activation='relu'),
          Dense(units=output_num_units, activation='softmax'),
        ])

# Defining parameters like optmizer, loss function and evaluating metric
model.compile(loss='categorical_crossentropy', # 
        optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
        metrics=['accuracy'])

# Defining early stopping callback.
# After epoch 50, training can be stopped if not improved in 'val_loss' is seen

cb_early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, min_delta=0.02) 
cb = keras.callbacks.TensorBoard(log_dir='early_stop', write_graph=False) 

history = model.fit(train_x, train_y, epochs=epochs, validation_split=0.2, callbacks=[cb, cb_early_stop])

Train on 15924 samples, validate on 3982 samples
Epoch 1/100
15924/15924 [==============================] - 52s 3ms/step - loss: 0.9358 - acc: 0.5767 - val_loss: 0.8415 - val_acc: 0.5939
Epoch 2/100
15924/15924 [==============================] - 64s 4ms/step - loss: 0.8364 - acc: 0.6063 - val_loss: 0.8105 - val_acc: 0.6246
Epoch 3/100
15924/15924 [==============================] - 48s 3ms/step - loss: 0.8170 - acc: 0.6191 - val_loss: 0.8027 - val_acc: 0.6421
Epoch 4/100
15924/15924 [==============================] - 66s 4ms/step - loss: 0.8002 - acc: 0.6317 - val_loss: 0.7873 - val_acc: 0.6602
Epoch 5/100
15924/15924 [==============================] - 48s 3ms/step - loss: 0.7903 - acc: 0.6387 - val_loss: 0.7704 - val_acc: 0.6524
Epoch 6/100
15924/15924 [==============================] - 52s 3ms/step - loss: 0.7725 - acc: 0.6534 - val_loss: 0.7818 - val_acc: 0.6552
Epoch 7/100
15924/15924 [==============================] - 48s 3ms/step - loss: 0.7617 - acc: 0.6596 - val_loss: 0.7442 - v

#### Training stopped on epoch 21 due to `val_loss` not improving from min_delta = 0.02